In [ ]:
from google.cloud import bigquery
from datetime import datetime
import os

# Configurações
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Users\geraldo.junior\DEV\Database_mkt\credenciais\big_query_key_geraldo-junior.json"
project_id = 'meta-vc-contabil'
source_dataset_id = 'marketing_db'
backup_dataset_id = f'marketing_db_backup_{datetime.now().strftime("%d%m%Y_%H%M")}'

# Inicializa o cliente do BigQuery
client = bigquery.Client(project=project_id)

# Cria o novo conjunto de dados para o backup
backup_dataset_ref = client.dataset(backup_dataset_id, project=project_id)
backup_dataset = bigquery.Dataset(backup_dataset_ref)
backup_dataset.location = 'US'  # Defina a localização do dataset
backup_dataset = client.create_dataset(backup_dataset)  # Cria o dataset

print(f'Conjunto de dados de backup criado: {backup_dataset_id}')

# Lista todas as tabelas no conjunto de dados original
tables = client.list_tables(source_dataset_id)

# Copia cada tabela para o novo conjunto de dados
for table in tables:
    source_table_id = f'{project_id}.{source_dataset_id}.{table.table_id}'
    destination_table_id = f'{project_id}.{backup_dataset_id}.{table.table_id}'
    
    job = client.copy_table(source_table_id, destination_table_id)
    job.result()  # Espera a conclusão do job

    print(f'Tabela {table.table_id} copiada para {destination_table_id}')

print('Backup concluído com sucesso!')